In [1]:
# ================================================ GET META VARIABLES ==================================================
import os
import glob
import json
import math
import joblib
import pandas as pd
import numpy as np
import seaborn as sns

from datetime import datetime
from IPython.display import display
from matplotlib.backends.backend_pdf import PdfPages

# ================================================ NEURAL NETWORK MODELS ===============================================
import tensorflow
tensorflow.random.set_seed(0)

from tensorflow import keras
from tensorflow.keras import backend as K, optimizers
from tensorflow.keras.layers import Activation, Conv2D, Dense, Dropout, Flatten, LSTM, Reshape
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from keras.callbacks import CSVLogger

In [2]:
import src.utils as utils
import src.load as load
import src.signal_processing as sig_process
import src.segmentation as segment
import src.create_features as feature_ex
import src.clean_dataset as cleaner
import src.training_testing_models as model_exe

In [3]:
CUR_DIR    = utils.get_full_path_directory(mode="main")
DATA_DIR   = utils.get_full_path_directory(mode="data")
CONFIG_DIR = utils.get_full_path_directory(mode="configs")
LOGS_DIR   = utils.get_full_path_directory(mode="logs")
TEST_DIR   = utils.get_full_path_directory(mode="tests")

In [4]:
with open(f"{CONFIG_DIR}/default.json", "r") as f:
    params        = json.load(f)["meta_params"]
    sampling_freq = params["sampling_freq"]
    window_size   = params["window_size"]
    prefix        = params["prefix"]
    n_splits      = params["n_splits"]
print("Configuration for meta_variables:", params)

Configuration for meta_variables: {'sampling_freq': 20, 'window_size': 64, 'prefix': 'wk8', 'n_splits': 10}


In [5]:
INFO = "wk_8(1)_8(2)_8(3)_8(4)_8(5)_8(6)_8(7)_XH_scSub"

In [6]:
separator  = "=========================================================="
Activities_index_name = pd.read_table(f"{CONFIG_DIR}/activity_labels.txt", sep=" ", header=None).to_dict()[0]
Activities_name_index = dict((y,x) for x,y in Activities_index_name.items())
Member_index_name     = pd.read_table(f"{CONFIG_DIR}/user_labels.txt", sep=" ", header=None).to_dict()[0]
Member_name_index     = dict((y,x) for x,y in Member_index_name.items())

In [7]:
PROCESSED_DIR = utils.prepare_dir(f"{DATA_DIR}/ProcessedData/{INFO}")

print(f"{PROCESSED_DIR}")

train_test_files_dic = utils.pickle_load(
    f"{PROCESSED_DIR}/{prefix}_freq_{sampling_freq}_train_test_files_dic_ws_{window_size}.pickle")

C:\Users\Acer User\Google Drive\RealDancingData\Training Data/ProcessedData/wk_8(1)_8(2)_8(3)_8(4)_8(5)_8(6)_8(7)_XH_scSub


In [8]:
visualize_provider    = True

<a id='Section3'></a>
## Step 3: Training and Testing ML Models

<a id='Section3.1'></a>
### Training and Testing ML Models

<a id='Section3.1.1'></a>
#### Importing datasets dictionary for training and testing

In [9]:
epochs = 1220
clf_name = "cnn"

In [14]:
def build_baseline(input_shape=(128, 6, 1), output_dim=6, lr=0.001):
    model = Sequential()
    model.add(Conv2D(int(window_size/2), kernel_size=(5, 1), input_shape=input_shape))
    model.add(Activation("relu"))
    model.add(Conv2D(int(window_size/2), kernel_size=(5, 1)))
    model.add(Activation("relu"))
    model.add(Conv2D(int(window_size/2), kernel_size=(5, 1)))
    model.add(Activation("relu"))
    model.add(Conv2D(int(window_size/2), kernel_size=(5, 1)))
    model.add(Activation("relu"))
    model.add(Flatten())
    model.add(Dense(window_size))
    model.add(Activation("relu"))
    model.add(Dropout(0.5, seed=0))
    model.add(Dense(window_size))
    model.add(Activation("relu"))
    model.add(Dropout(0.5, seed=1))
    model.add(Dense(output_dim))
    model.add(Activation("softmax"))
    model.compile(
            loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=lr), metrics=["accuracy"]
    )
    return model

In [11]:
files = train_test_files_dic[3.2]

models = []
classifiers_summary_datasets = pd.DataFrame()
cv_df_meta_datasets = pd.DataFrame()

classifiers_summary = pd.DataFrame()
results = {}

X_train, X_test, y_train, y_test = files[0], files[1], files[2], files[3]
print(f"The dataset shapes are: {X_train.shape}, {y_train.shape}, {X_test.shape}, {y_test.shape}")

classifiers_summary = pd.DataFrame()

num_classes = len(np.unique(y_train))

print(f"{clf_name} started training....")

params_model = None

test_preds = np.zeros((1, X_test.shape[0], num_classes))

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

The dataset shapes are: (77714, 64, 6, 1), (77714,), (8463, 64, 6, 1), (8463,)
cnn started training....
training with params: {'batch_size': 128, 'epochs': 10000, 'lr': 0.001, 'verbose': 0}


In [18]:
with open(f"{CONFIG_DIR}/default.json", "r") as f:
    params_model = json.load(f)[f"{clf_name}_params"]
print(f"training with params: {params_model}")

model = build_baseline(input_shape=X_train.shape[1:], output_dim=y_train.shape[1], lr=params_model["lr"])

fit = model.fit(X_train, y_train, batch_size=params_model["batch_size"],
                epochs=epochs, verbose=1,
                validation_data=(X_test, y_test))

model.save(f"trained_model_dataset_{key}_model_{clf_name}_fold({fold_id})")

training with params: {'batch_size': 128, 'epochs': 10000, 'lr': 0.001, 'verbose': 0}
Epoch 1/1220
170/608 [=======>......................] - ETA: 36s - loss: 1.2010 - accuracy: 0.5872

KeyboardInterrupt: 